In [ ]:
import generation as gen
import prediction as prediction
import utils
import files
import os
import numpy as np
from tqdm.autonotebook import tqdm

In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd
from matplotlib import cm

In [ ]:
size=19
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Times',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'font.size': size,
    'axes.labelsize':size,
    'axes.titlesize':size,
    'figure.titlesize':size,
    'xtick.labelsize':size,
    'ytick.labelsize':size,
    'legend.fontsize':size,
})

In [ ]:
alpha = 0.1
d = 10
phi = 0.8
regression = 'Linear'
n_rep = 2
beta = np.array([1, 2, -1, 3, -0.5, -1, 0.3, 1.7, 0.4, -0.3])

train_size = 500
cal_size = 250
params_test = {'iid':{'test_size': 2000}, 
               'fixed_nb_sample_pattern':{'nb_sample_pattern': 100}, 
               'fixed_nb_sample_pattern_size':{'nb_sample_pattern': 100}}
params_test = gen.process_test(params_test, d=d)

params_reg = {'regression':regression, 'beta': beta, 'phi': phi}

params_noise = {'noise':'Gaussian'}

prob_missing = 0.2
var_missing = np.full(d, 1)
params_missing = {'prob_missing':prob_missing, 'var_missing':var_missing, 'mechanism': 'MCAR'}

In [ ]:
methods = ['QR_TrainCal','CQR','CQR_Masking_Cal']

basemodel = 'NNet'
mask = 'Yes'
protection = 'No'
imputation = 'iterative_ridge'

name_pipeline_to_plot = []
for method in methods: 
    if method == 'CQR_Masking_Cal':
        name_temp = files.get_name_method(method, basemodel, mask=mask, protection=protection, subset=True)
        if not name_temp in name_pipeline_to_plot:
            name_pipeline_to_plot.append(name_temp)
            
    name_temp = files.get_name_method(method, basemodel, mask=mask, protection=protection, subset=False)
    if not name_temp in name_pipeline_to_plot:
        name_pipeline_to_plot.append(name_temp)
    
current_pipeline = method+'_'+basemodel

In [ ]:
dict_cov = dict.fromkeys(name_pipeline_to_plot)
dict_len = dict.fromkeys(name_pipeline_to_plot)

for pipeline in name_pipeline_to_plot:
    dict_cov[pipeline] = {}
    dict_len[pipeline] = {}

In [ ]:
keys_pattern = np.arange(d)

In [ ]:
impute_inf = True
replace_inf = True

In [ ]:
key = -1

nb_boxplot = len(keys_pattern)+1

name_method = []

for pipeline in tqdm(name_pipeline_to_plot):

    name_method = np.append(name_method, '_'.join([imputation, pipeline]))

    data, results = utils.get_data_results(pipeline, train_size, cal_size, params_test, n_rep, d=d, imputation=imputation,
                                           params_reg=params_reg, params_noise=params_noise, params_missing=params_missing,
                                           parent_results='results', parent_data='data', extension='xz')

    contains, lengths = utils.compute_PI_metrics(data, results, 'iid')
    
    if replace_inf:
        max_y_train = np.max(data['Y']['Train'], axis=1)
        max_y_cal = np.max(data['Y']['Cal'], axis=1)
        min_y_train = np.min(data['Y']['Train'], axis=1)
        min_y_cal = np.min(data['Y']['Cal'], axis=1)
        max_length_traincal = np.maximum(max_y_train, max_y_cal)-np.minimum(min_y_train, min_y_cal)
        for k in range(n_rep):
            idx_inf = np.where(np.isinf(lengths[k,:]))[0]
            if len(idx_inf)>0:
                lengths[k,:][idx_inf] = max_length_traincal[k]
    
    metrics = utils.compute_metrics_cond(n_rep, data, results, 'fixed_nb_sample_pattern_size', cond='Pattern_Size',
                                         replace_inf=replace_inf)
    
    dict_cov[pipeline][key] = np.mean(contains, axis=1)
    dict_len[pipeline][key] = np.mean(lengths, axis=1)

    #key += 1

    for key_pattern in keys_pattern:

        dict_cov[pipeline][key_pattern] = metrics[key_pattern]['avg_cov']
        dict_len[pipeline][key_pattern] = metrics[key_pattern]['avg_len']

        #key += 1

In [ ]:
import functools

In [ ]:
if 'phi' in params_reg:
    phi = params_reg['phi']
else:
    phi = 0.8
cov = np.full((d,d),phi)+(1-phi)*np.eye(d)

In [ ]:
len_oracle_marginal = []
M_test = data['M']['Test']['iid']
test_size = M_test.shape[1]
for i in range(n_rep):
    
    M_test_i = M_test[i,:,:]
    patterns = np.unique(M_test_i, axis=0)
    oracles_len_per_pattern = list(map(functools.partial(prediction.oracle_len_pattern, beta=beta, cov=cov, alpha=0.1), patterns))

    len_oracle = np.empty(test_size)
    
    for idp, pattern in enumerate(patterns):
        pattern_id = utils.pattern_to_id(pattern.astype(int))
        M_test_id = list(map(utils.pattern_to_id, M_test_i.astype(int)))
        len_oracle[np.where(np.array(M_test_id) == pattern_id)] = oracles_len_per_pattern[idp]
    len_oracle_marginal = np.append(len_oracle_marginal, np.mean(len_oracle))

In [ ]:
len_oracle = {}
patterns_by_size = dict.fromkeys(np.arange(0,d))
for k in range(d):
    patterns_by_size[k] = []
patterns_id = np.arange(0, 2**d-1)
for pattern_id in patterns_id:
    vec_pattern = utils.bin_to_vec(bin(pattern_id), d)
    size_pattern = utils.pattern_to_size(vec_pattern)
    patterns_by_size[size_pattern] = np.append(patterns_by_size[size_pattern], pattern_id)
for k in range(d):
    list_len = []
    for pattern_id in patterns_by_size[k]:
        vec_pattern = utils.bin_to_vec(bin(np.int(pattern_id)), d)
        list_len = np.append(list_len, prediction.oracle_len_pattern(vec_pattern, beta, cov))
    len_oracle[k] = np.mean(list_len)

In [ ]:
import seaborn as sns

In [ ]:
dict_methods = {'QR_TrainCal_NNet_Mask': 'QR',
                'QR_NNet_Mask': 'QR',
                'CQR_NNet_Mask': 'CQR', 
                'CQR_Masking_Cal_subset_NNet_Mask': 'CQR-MDA-Exact',
                'CQR_Masking_Cal_NNet_Mask': 'CQR-MDA-Nested'}

In [ ]:
fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2, 4, figsize=(15,6), sharey='row')

name_ticks = list(map(utils.name_tick, name_method))
colors_palette = sns.color_palette("husl", nb_boxplot)
colors = colors_palette 

ax = [ax1, ax2, ax3, ax4]
nb_subplots = len(ax)
for axi in ax:
    axi.axhline(1-alpha, color='black', ls='--')

axl = [ax5, ax6, ax7, ax8]
    
for idp, pipeline in enumerate(name_pipeline_to_plot):
    
    ax[idp].set_title(dict_methods[pipeline])
    
    box = ax[idp].violinplot(dict_cov[pipeline].values(), showmeans=True, showextrema=False)#, quantiles=[[0.25, 0.75]]*nb_boxes)#, patch_artist=True)
    for pc,color in zip(box['bodies'], colors):
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    box['cmeans'].set_color('black')
    
    box = axl[idp].violinplot(dict_len[pipeline].values(), showmeans=True, showextrema=False)#, quantiles=[[0.25, 0.75]]*nb_boxes)#, patch_artist=True)
    for pc,color in zip(box['bodies'], colors):
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
        pc.set_alpha(1)
    box['cmeans'].set_color('black')
    
idx = np.arange(d+1)
idy = np.append([np.mean(len_oracle_marginal)], np.array(list(len_oracle.values())))

for axi in axl:
    axi.scatter(idx+1, idy, color=colors, zorder=2, marker='*', s=100, edgecolor='black')

for axi in ax:
    axi.set_xticks([])
    
name_ticks_missing = []
for k in range(d):
    name_ticks_missing = np.append(name_ticks_missing, str(k)+r' \texttt{NA}')
name_ticks = np.append(['Marg.'], name_ticks_missing)

for axi in axl:
    ticks = np.arange(0,d+1)
    axi.set_xticks(ticks+1)
    axi.set_xticklabels(name_ticks, rotation=70)

ax1.set_ylabel('Average coverage')
ax5.set_ylabel('Average length')

ax5.legend(handles = [mlines.Line2D([], [], marker="*", linestyle='None', markersize=15, markeredgecolor='black', markerfacecolor='White')],
           labels=['Oracle length'], loc='upper left', handletextpad=10**(-60))

fig.tight_layout()

name_plot = 'plots/synthetic/Linear_d_'+str(d)+'_NA_'+str(prob_missing)+'_imputation_'+str(imputation)+'_basemodel_'+basemodel
if mask == 'Yes':
    name_plot = name_plot + '_mask' 
name_plot = name_plot + '_train_'+str(train_size) + '_cal_'+str(cal_size) +'_rep_'+str(n_rep)
if mask == 'No':
    name_plot = name_plot+'_nomask'
if impute_inf:
    name_plot = name_plot+'_replaceinf'
plt.savefig(name_plot+'.pdf',bbox_inches='tight', dpi=300)

plt.show()